In [6]:
import requests
import pandas as pd

API_KEY = "NGELwQY8b2XQcUu8sAs9a96T9COtSsacKzI0FK4gJmS0qgqGjTRWgRBfNG3sfwt0"
BASE_URL = "https://api.collegefootballdata.com/plays"

years = [2022]
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

params_list = [{'year': year, 'week': week, 'classification': 'fbs'} for year in years for week in weeks]

headers = {'Authorization': f'Bearer {API_KEY}'}

def get_data_from_api(params):
    res = requests.get(BASE_URL, params=params, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        print(f"Error fetching data for year {params['year']} week {params['week']}: {res.status_code}")
        return None

data = []
for params in params_list:
    week_data = get_data_from_api(params)
    if week_data:
        for play in week_data:
            play['week'] = params['week']
            play['year'] = params['year']
        data.extend(week_data)

df = pd.DataFrame(data)

print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152806 entries, 0 to 152805
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  152806 non-null  object 
 1   offense             152806 non-null  object 
 2   offense_conference  152806 non-null  object 
 3   defense             152806 non-null  object 
 4   defense_conference  152806 non-null  object 
 5   home                152806 non-null  object 
 6   away                152806 non-null  object 
 7   offense_score       152806 non-null  int64  
 8   defense_score       152806 non-null  int64  
 9   game_id             152806 non-null  int64  
 10  drive_id            152806 non-null  object 
 11  drive_number        152806 non-null  int64  
 12  play_number         152806 non-null  int64  
 13  period              152806 non-null  int64  
 14  clock               152806 non-null  object 
 15  offense_timeouts    152394 non-nul

In [7]:
df["distance_to_goal"] = 100 - df["yard_line"]
df['red_zone'] = (df['yards_to_goal'] <= 20).astype(int)
df['down_yards_interaction'] = df['down'] * df['yards_to_goal']
df['distance_ratio'] = df['distance_to_goal'] / (df['yards_to_goal'] + 1e-5)
df['score_diff_start'] = df['offense_score'] - df['defense_score']
df['close_score'] = (df['score_diff_start'].abs() < 10).astype(int)
print(len(df['offense'].unique()))
print(df.info())

228
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152806 entries, 0 to 152805
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      152806 non-null  object 
 1   offense                 152806 non-null  object 
 2   offense_conference      152806 non-null  object 
 3   defense                 152806 non-null  object 
 4   defense_conference      152806 non-null  object 
 5   home                    152806 non-null  object 
 6   away                    152806 non-null  object 
 7   offense_score           152806 non-null  int64  
 8   defense_score           152806 non-null  int64  
 9   game_id                 152806 non-null  int64  
 10  drive_id                152806 non-null  object 
 11  drive_number            152806 non-null  int64  
 12  play_number             152806 non-null  int64  
 13  period                  152806 non-null  int64  
 14  clock           

In [8]:
import ast

def extract_time(time_str):
    try:
        time_dict = ast.literal_eval(time_str) 
        minutes = time_dict.get("minutes", 0)
        seconds = time_dict.get("seconds", 0)
        return minutes, seconds
    except:
        return 0, 0

df[['start_time_minutes', 'start_time_seconds']] = df['wallclock'].apply(lambda x: pd.Series(extract_time(x)))

df['start_time_in_seconds'] = df['start_time_minutes']*60 + df['start_time_seconds']

df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152806 entries, 0 to 152805
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      152806 non-null  object 
 1   offense                 152806 non-null  object 
 2   offense_conference      152806 non-null  object 
 3   defense                 152806 non-null  object 
 4   defense_conference      152806 non-null  object 
 5   home                    152806 non-null  object 
 6   away                    152806 non-null  object 
 7   offense_score           152806 non-null  int64  
 8   defense_score           152806 non-null  int64  
 9   game_id                 152806 non-null  int64  
 10  drive_id                152806 non-null  object 
 11  drive_number            152806 non-null  int64  
 12  play_number             152806 non-null  int64  
 13  period                  152806 non-null  int64  
 14  clock               

(None,
                    id     offense offense_conference     defense  \
 0  401403853101849903  Vanderbilt                SEC     Hawai'i   
 1  401403853101849905     Hawai'i      Mountain West  Vanderbilt   
 2  401403853101849906     Hawai'i      Mountain West  Vanderbilt   
 3  401403853101849907     Hawai'i      Mountain West  Vanderbilt   
 4  401403853101849908     Hawai'i      Mountain West  Vanderbilt   
 
   defense_conference     home        away  offense_score  defense_score  \
 0      Mountain West  Hawai'i  Vanderbilt              0              0   
 1                SEC  Hawai'i  Vanderbilt              0              0   
 2                SEC  Hawai'i  Vanderbilt              0              0   
 3                SEC  Hawai'i  Vanderbilt              0              0   
 4                SEC  Hawai'i  Vanderbilt              0              0   
 
      game_id  ...  year  distance_to_goal  red_zone  down_yards_interaction  \
 0  401403853  ...  2022               

In [9]:
from sklearn.preprocessing import LabelEncoder

matchup_encoder = LabelEncoder()
df["offense_defense_matchup_encoded"] = matchup_encoder.fit_transform(df["offense"] + "_" + df["defense"])
df['offense_idx'] = matchup_encoder.fit_transform(df['offense'])
df['defense_idx'] = matchup_encoder.fit_transform(df['defense'])
df['offense_conference_idx'] = matchup_encoder.fit_transform(df['offense_conference'])
df['defense_conference_idx'] = matchup_encoder.fit_transform(df['defense_conference'])
df["offense_defense_matchup"] = df["offense"] + "_" + df["defense"]
df.info()

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152806 entries, 0 to 152805
Data columns (total 44 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   id                               152806 non-null  object 
 1   offense                          152806 non-null  object 
 2   offense_conference               152806 non-null  object 
 3   defense                          152806 non-null  object 
 4   defense_conference               152806 non-null  object 
 5   home                             152806 non-null  object 
 6   away                             152806 non-null  object 
 7   offense_score                    152806 non-null  int64  
 8   defense_score                    152806 non-null  int64  
 9   game_id                          152806 non-null  int64  
 10  drive_id                         152806 non-null  object 
 11  drive_number                     152806 non-null  int64  
 12  pl

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,game_id,...,close_score,start_time_minutes,start_time_seconds,start_time_in_seconds,offense_defense_matchup_encoded,offense_idx,defense_idx,offense_conference_idx,defense_conference_idx,offense_defense_matchup
0,401403853101849903,Vanderbilt,SEC,Hawai'i,Mountain West,Hawai'i,Vanderbilt,0,0,401403853,...,1,0,0,0,1563,209,65,19,13,Vanderbilt_Hawai'i
1,401403853101849905,Hawai'i,Mountain West,Vanderbilt,SEC,Hawai'i,Vanderbilt,0,0,401403853,...,1,0,0,0,492,65,209,13,19,Hawai'i_Vanderbilt
2,401403853101849906,Hawai'i,Mountain West,Vanderbilt,SEC,Hawai'i,Vanderbilt,0,0,401403853,...,1,0,0,0,492,65,209,13,19,Hawai'i_Vanderbilt
3,401403853101849907,Hawai'i,Mountain West,Vanderbilt,SEC,Hawai'i,Vanderbilt,0,0,401403853,...,1,0,0,0,492,65,209,13,19,Hawai'i_Vanderbilt
4,401403853101849908,Hawai'i,Mountain West,Vanderbilt,SEC,Hawai'i,Vanderbilt,0,0,401403853,...,1,0,0,0,492,65,209,13,19,Hawai'i_Vanderbilt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152801,401404145106290207,Army,FBS Independents,Navy,American Athletic,Army,Navy,17,17,401404145,...,1,0,0,0,107,13,114,9,1,Army_Navy
152802,401404145106290208,Army,FBS Independents,Navy,American Athletic,Army,Navy,17,17,401404145,...,1,0,0,0,107,13,114,9,1,Army_Navy
152803,401404145106290209,Army,FBS Independents,Navy,American Athletic,Army,Navy,17,17,401404145,...,1,0,0,0,107,13,114,9,1,Army_Navy
152804,401404145106290210,Army,FBS Independents,Navy,American Athletic,Army,Navy,20,17,401404145,...,1,0,0,0,107,13,114,9,1,Army_Navy


In [ ]:
df.to_csv('../data/plays2022.csv', index=False)